# Exercise 2

In [3]:
%config Completer.use_jedi = False
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [4]:
spark

## 1. Read the file and convert to structured data.  

### DataFrame mode

In [12]:
df = spark.read.json('students.txt')
df.cache()
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- age: long (nullable = true)
 |-- email: string (nullable = true)
 |-- grade: double (nullable = true)
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)



### SQL mode

In [21]:
df.createOrReplaceTempView('students')
spark.sql('select * from students limit 1').printSchema()

root
 |-- _id: string (nullable = true)
 |-- age: long (nullable = true)
 |-- email: string (nullable = true)
 |-- grade: double (nullable = true)
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)



## 2. Obtain the top 5 students with more amount of grades.  

### DataFrame mode

In [43]:
from pyspark.sql.functions import *
df.select('*').orderBy(desc('grade')).limit(5).show()

+--------------------+---+--------------------+-----+-------+----------------+
|                 _id|age|               email|grade|   name|         surname|
+--------------------+---+--------------------+-----+-------+----------------+
|9e27157e-062b-47f...| 22|LaiayVictoria@gma...| 10.0|   Laia|   Segovia Lopez|
|1b36d010-64c0-4aa...| 23|Elena1993@outlook...| 9.99|  Elena| Sanchez Gisbert|
|afae424a-3c2b-43f...| 25|Gomez.Segarra@hot...| 9.99|   Sara|   Gomez Segarra|
|cc8f7297-5920-431...| 25|David.Pascual@yah...| 9.98|  David|Pascual Gonzalez|
|8a4da0df-2988-4f9...| 19|Esteban1997@gmail...| 9.98|Esteban|   Comas Segovia|
+--------------------+---+--------------------+-----+-------+----------------+



### SQL mode

In [30]:
spark.sql('select * from students order by grade desc limit 5').show()

+--------------------+---+--------------------+-----+-------+----------------+
|                 _id|age|               email|grade|   name|         surname|
+--------------------+---+--------------------+-----+-------+----------------+
|9e27157e-062b-47f...| 22|LaiayVictoria@gma...| 10.0|   Laia|   Segovia Lopez|
|1b36d010-64c0-4aa...| 23|Elena1993@outlook...| 9.99|  Elena| Sanchez Gisbert|
|afae424a-3c2b-43f...| 25|Gomez.Segarra@hot...| 9.99|   Sara|   Gomez Segarra|
|cc8f7297-5920-431...| 25|David.Pascual@yah...| 9.98|  David|Pascual Gonzalez|
|8a4da0df-2988-4f9...| 19|Esteban1997@gmail...| 9.98|Esteban|   Comas Segovia|
+--------------------+---+--------------------+-----+-------+----------------+



## 3. Calculate max and mean of grades and obtain the best 5 students of both groups.  

### DataFrame mode

#### Max and mean of grades

In [65]:
df.select(max('grade').alias('max_grade'), mean('grade').alias('mean_grades')).show()

+---------+-----------------+
|max_grade|      mean_grades|
+---------+-----------------+
|     10.0|7.954802000000002|
+---------+-----------------+



In [74]:
df.groupBy('name', 'surname')\
    .agg(mean('grade').alias('mean_grade'))\
    .orderBy(desc('mean_grade'))\
    .limit(5).show()

+--------+----------------+----------+
|    name|         surname|mean_grade|
+--------+----------------+----------+
|    Laia|   Segovia Lopez|      10.0|
|   Elena| Sanchez Gisbert|      9.99|
| Esteban|   Comas Segovia|      9.98|
|   David|Pascual Gonzalez|      9.98|
|Fernando| Bermejo Gisbert|      9.98|
+--------+----------------+----------+



### SQL mode

#### Max and mean of grades

In [46]:
spark.sql('select max(grade) as max_grade, mean(grade) as mean_grades from students').show()

+---------+-----------------+
|max_grade|      mean_grades|
+---------+-----------------+
|     10.0|7.954802000000002|
+---------+-----------------+



#### Top 5 students with best mean grade

In [60]:
spark.sql("""select 
                name, surname, mean(grade) as mean_grades 
            from 
                students 
            group by name, surname 
            order by mean_grades desc limit 5""").show()

+--------+----------------+-----------+
|    name|         surname|mean_grades|
+--------+----------------+-----------+
|    Laia|   Segovia Lopez|       10.0|
|   Elena| Sanchez Gisbert|       9.99|
| Esteban|   Comas Segovia|       9.98|
|   David|Pascual Gonzalez|       9.98|
|Fernando| Bermejo Gisbert|       9.98|
+--------+----------------+-----------+



## 4. Filter students older than 20 years old. 

### DataFrame mode

In [71]:
df.filter('age > 20').orderBy('age').show()

+--------------------+---+--------------------+-----+--------+----------------+
|                 _id|age|               email|grade|    name|         surname|
+--------------------+---+--------------------+-----+--------+----------------+
|8953f082-72be-45a...| 21|DavidyBerta@gmail...| 7.81|   David|Gonzalez Sanchez|
|0e64a7e9-0778-4b7...| 21|DavidyHugo@hotmai...| 7.39|   David| Gonzalez Merino|
|24bff53f-5551-48c...| 21|  Jose1995@gmail.com| 7.66|    Jose|   Bermejo Lopez|
|fbdf66dc-49da-467...| 21| IkerySara@gmail.com| 7.77|    Iker|   Seco Coronado|
|3d774c8c-4ddf-444...| 21|Tomas1995@hotmail...| 8.73|   Tomas|    Grande Comas|
|971deb27-156d-4cd...| 21|Aznar.Pascual@idx...|  8.6|Santiago|   Aznar Pascual|
|1a2f1928-9060-4c9...| 21|JoseyJuana@gmail.com|  7.2|    Jose|       Tous Tous|
|bf9d3e1f-ebdc-486...| 21|Agustin1995@hotma...| 9.62| Agustin|  Sanchez Cuesta|
|3dfb89d9-5856-482...| 21|Sara1995@hotmail.com| 7.21|    Sara|     Rajoy Agudo|
|57f94074-a49a-4e6...| 21|Sanchez.Crespo

### SQL mode

In [68]:
spark.sql("""select * from students where age > 20 order by age""").show()

+--------------------+---+--------------------+-----+--------+----------------+
|                 _id|age|               email|grade|    name|         surname|
+--------------------+---+--------------------+-----+--------+----------------+
|8953f082-72be-45a...| 21|DavidyBerta@gmail...| 7.81|   David|Gonzalez Sanchez|
|0e64a7e9-0778-4b7...| 21|DavidyHugo@hotmai...| 7.39|   David| Gonzalez Merino|
|24bff53f-5551-48c...| 21|  Jose1995@gmail.com| 7.66|    Jose|   Bermejo Lopez|
|fbdf66dc-49da-467...| 21| IkerySara@gmail.com| 7.77|    Iker|   Seco Coronado|
|3d774c8c-4ddf-444...| 21|Tomas1995@hotmail...| 8.73|   Tomas|    Grande Comas|
|971deb27-156d-4cd...| 21|Aznar.Pascual@idx...|  8.6|Santiago|   Aznar Pascual|
|1a2f1928-9060-4c9...| 21|JoseyJuana@gmail.com|  7.2|    Jose|       Tous Tous|
|bf9d3e1f-ebdc-486...| 21|Agustin1995@hotma...| 9.62| Agustin|  Sanchez Cuesta|
|3dfb89d9-5856-482...| 21|Sara1995@hotmail.com| 7.21|    Sara|     Rajoy Agudo|
|57f94074-a49a-4e6...| 21|Sanchez.Crespo

## 5. Calculate the average of grades for each students year.

### DataFrame mode

In [78]:
df.groupBy('age').agg(mean('grade').alias('mean_grade')).orderBy('age').show()

+---+------------------+
|age|        mean_grade|
+---+------------------+
| 18| 7.935931528662416|
| 19|7.9874188034187945|
| 20| 7.920192791282483|
| 21|7.9860449050086375|
| 22| 7.916797804208606|
| 23|7.9727236971484805|
| 24| 7.999281364190009|
| 25|7.9337354651162775|
| 26| 7.903791102514502|
| 27|  7.92304878048781|
| 28| 7.921007194244603|
| 29| 8.041525423728814|
| 30| 8.114893617021275|
| 31| 8.014242424242425|
| 32|             8.091|
| 33| 8.180588235294119|
| 34|7.9399999999999995|
| 35|             7.698|
| 36|              7.68|
| 37|              8.11|
+---+------------------+



### SQL mode

In [76]:
spark.sql("""
    select age, mean(grade) as mean_grade
    from students
    group by age
    order by age
""").show()

+---+------------------+
|age|        mean_grade|
+---+------------------+
| 18| 7.935931528662416|
| 19|7.9874188034187945|
| 20| 7.920192791282483|
| 21|7.9860449050086375|
| 22| 7.916797804208606|
| 23|7.9727236971484805|
| 24| 7.999281364190009|
| 25|7.9337354651162775|
| 26| 7.903791102514502|
| 27|  7.92304878048781|
| 28| 7.921007194244603|
| 29| 8.041525423728814|
| 30| 8.114893617021275|
| 31| 8.014242424242425|
| 32|             8.091|
| 33| 8.180588235294119|
| 34|7.9399999999999995|
| 35|             7.698|
| 36|              7.68|
| 37|              8.11|
+---+------------------+



## 6. Add a column named excellent which condition is its student grade > 9.5. 

### DataFrame mode

In [90]:
df.withColumn('excellent', when(col('grade') > 9.5, 'GENIO').otherwise('TONTO')).show()

+--------------------+---+--------------------+-----+--------+-----------------+---------+
|                 _id|age|               email|grade|    name|          surname|excellent|
+--------------------+---+--------------------+-----+--------+-----------------+---------+
|33a624e7-e6f1-40b...| 23|Valeria.Sebastian...| 7.56| Valeria| Sebastian Garcia|    TONTO|
|2cd47675-43f3-415...| 23|Sanchez.Abascal@g...| 8.16|    Emma|  Sanchez Abascal|    TONTO|
|594ea4e7-75e3-456...| 20|Sarabia.Lopez@gma...| 8.22| Agustin|    Sarabia Lopez|    TONTO|
|3b521244-d2d4-40b...| 25|MartinaySebastian...| 7.67| Martina|Corominas Sarabia|    TONTO|
|e6f52130-362f-4a5...| 19|DavidyValeria@gma...| 7.45|   David|   Miranda Grande|    TONTO|
|cee04454-f6ea-48b...| 20|Lopez.Bernal@outl...| 7.35|    Laia|     Lopez Bernal|    TONTO|
|6e5b75cd-0d5f-41f...| 22|MarcosySantiago@h...|  6.8|  Marcos|     Garcia Aznar|    TONTO|
|47435195-80b1-473...| 18|Judith.Garcia@gma...|  9.1|  Judith|      Garcia Cruz|    TONTO|

### SQL mode

In [87]:
spark.sql("""
    select *,
        case 
            when grade > 9.5 then 'GENIO'
            else 'TONTO'
        end as excellent
    from students
""").show()

+--------------------+---+--------------------+-----+--------+-----------------+---------+
|                 _id|age|               email|grade|    name|          surname|excellent|
+--------------------+---+--------------------+-----+--------+-----------------+---------+
|33a624e7-e6f1-40b...| 23|Valeria.Sebastian...| 7.56| Valeria| Sebastian Garcia|    TONTO|
|2cd47675-43f3-415...| 23|Sanchez.Abascal@g...| 8.16|    Emma|  Sanchez Abascal|    TONTO|
|594ea4e7-75e3-456...| 20|Sarabia.Lopez@gma...| 8.22| Agustin|    Sarabia Lopez|    TONTO|
|3b521244-d2d4-40b...| 25|MartinaySebastian...| 7.67| Martina|Corominas Sarabia|    TONTO|
|e6f52130-362f-4a5...| 19|DavidyValeria@gma...| 7.45|   David|   Miranda Grande|    TONTO|
|cee04454-f6ea-48b...| 20|Lopez.Bernal@outl...| 7.35|    Laia|     Lopez Bernal|    TONTO|
|6e5b75cd-0d5f-41f...| 22|MarcosySantiago@h...|  6.8|  Marcos|     Garcia Aznar|    TONTO|
|47435195-80b1-473...| 18|Judith.Garcia@gma...|  9.1|  Judith|      Garcia Cruz|    TONTO|